In [1]:
#Description: Predicting Ethereum Price with Python and Machine Learning

In [2]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
import matplotlib.pyplot as plt
import requests
plt.style.use('fivethirtyeight')

# Import required libraries
import requests
from datetime import datetime, timedelta

In [3]:
# Define the API endpoint and parameters
endpoint = "https://api.pro.coinbase.com/products/ETH-USD/candles"
params = {
        "start": (datetime.now() - timedelta(days=365)).isoformat(),  # Start date (1 year ago)
    "end": datetime.now().isoformat(),  # End date (today)
    "granularity": 86400,  # Daily granularity (86400 seconds)
}



In [4]:
# Send a GET request to the API
response = requests.get(endpoint, params=params)

In [5]:

# Check if the request was successful
if response.status_code == 200:
    data = response.json()['prices']
    
    # Convert the data to a Pandas DataFrame
    df = pd.DataFrame(data, columns=['timestamp', 'price'])
    df['timestamp'] = [datetime.fromtimestamp(ts / 1000) for ts in df['timestamp']]
    df = df.set_index('timestamp')
    
    # Print the first few rows of the DataFrame
    print(df.head())
else:
    print(f'Error: {response.status_code}')
    
    
    
    #Debug Code due to error code: 400 Bad Request

Error: 400
